In [1]:
import tensorflow as tf
import tensorflow.keras.layers as tfkl
import numpy as np 
import matplotlib.pyplot as plt
from tqdm import tqdm

2021-11-13 14:32:37.284846: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


Cargo el archivo de texto con el que voy a trabajar. Faltan algunas tareas por realizar, como: 
- Limpiar algunas palabras que no son parte de las letras (como la palabra estribillo)


In [2]:
#filepath = '../data/folclore-web/clean_dataset.txt'
filepath = '../data/chamame-web/clean_dataset.txt'
f = open(filepath)
folclore_dataset = f.read()
print(folclore_dataset[:500])

niangapiri silvestre luz del arenal 
tibio rubor del verde gris de la niniez 


mi resplandor mi atardecer mi taragi 
y el cielo aqui de tu mirar niangapiri 


anduve mirando el pueblo buscandome 
y fluyes como mi sangre de chamame 
enciende tu pie desnudo sobre mi piel 
un rio de primavera ternura y miel 
a orillas de tu sonrisa yo soy feliz 
no quiero partir de nuevo niangapiri 


la luna se ardio en el rio niangapiri 
y vuela a lo mas profundo de tu vivir 
alli donde es todo aroma va mi canci


Ahora codifico a cada caracter como un numero entero. Para eso se usa el *Tokenizer*. Esto lo que hace es primero identificar todos los caracteres presentes, y luego asignarle un codigo o Token a cada uno. Empieza desde 1 hasta el numero de caracteres diferentes que encuentre en el dataset. Se debe tener en cuenta que el tokenizer convierte todo el texto en lowercase. Se puede indicarle lower=False

In [3]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True) #default es word level encoding! 
tokenizer.fit_on_texts(folclore_dataset)

Con esto, el tokenizer puede codificar texto en forma de tokes o IDs, y viceversa. Ademas, nos dice cuantos caracteres diferentes hay, y el numero de caracteres en el texto. 

In [4]:
tokenizer.texts_to_sequences(['Hola!'])

[[22, 4, 10, 3]]

In [5]:
tokenizer.sequences_to_texts([[23, 4, 9, 3, 45]])

['j o i a']

In [6]:
tokenizer.word_index

{' ': 1,
 'e': 2,
 'a': 3,
 'o': 4,
 'n': 5,
 'r': 6,
 's': 7,
 '\n': 8,
 'i': 9,
 'l': 10,
 'u': 11,
 'd': 12,
 't': 13,
 'c': 14,
 'm': 15,
 'p': 16,
 'v': 17,
 'y': 18,
 'q': 19,
 'g': 20,
 'b': 21,
 'h': 22,
 'j': 23,
 'z': 24,
 'f': 25,
 'k': 26,
 'x': 27}

In [7]:
max_id = len(tokenizer.word_index) # Cantiad de caracteres distintos
print('Cantidad de caracteres diferentes: ', max_id)
dataset_size = tokenizer.document_count
print('Cantidad de caracteres en el dataset: ',dataset_size) # Cantidad de caracteres en el dataset

Cantidad de caracteres diferentes:  27
Cantidad de caracteres en el dataset:  88204


A fines practicos, voy a codificar todo el dataset para que cada caracter este representado por su ID. Voy a restar uno a la codificación para ir de 0 a 81 en lugar de 1 a 82.

In [8]:
encoded = np.array(tokenizer.texts_to_sequences(folclore_dataset)) - 1
len(encoded)

88204

divido los datos para entrenamiento, validacion y prueba. 

In [9]:
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

2021-11-13 14:32:47.990551: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-11-13 14:32:48.023730: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-13 14:32:48.024022: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1060 6GB computeCapability: 6.1
coreClock: 1.7715GHz coreCount: 10 deviceMemorySize: 5.93GiB deviceMemoryBandwidth: 178.99GiB/s
2021-11-13 14:32:48.024040: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-11-13 14:32:48.026949: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2021-11-13 14:32:48.026978: I tensorflow/stream_executo

In [10]:
#visualizo algunos datos. Son numeros! 
for i in dataset.take(10):
    print(i)

tf.Tensor([4], shape=(1,), dtype=int64)
tf.Tensor([8], shape=(1,), dtype=int64)
tf.Tensor([2], shape=(1,), dtype=int64)
tf.Tensor([4], shape=(1,), dtype=int64)
tf.Tensor([19], shape=(1,), dtype=int64)
tf.Tensor([2], shape=(1,), dtype=int64)
tf.Tensor([15], shape=(1,), dtype=int64)
tf.Tensor([8], shape=(1,), dtype=int64)
tf.Tensor([5], shape=(1,), dtype=int64)
tf.Tensor([8], shape=(1,), dtype=int64)


Corto el dataset en ventanas, que son las secuencias con las que voy a entrenar. Esto lo hacemos para tener secuencias mas cortas, si entreo a la red con esta secuencia asi como esta, se volveria tan profunda y pesada que seria imposible entrenarla en una PC. Basicamente, lo que hacemos es truncar el proceso de backpropagation en el tiempo. El tamaño de los vectores temporales es un HIPERPARAMETRO MUY IMPORTANTE, porque determina la memoria que tendra este modelo. A menor valor, se entrena mas facil pero tiene menos memoria.

In [11]:
n_steps = 100 # largo de cada ventanita
window_length = n_steps + 1 # ya que el target es la entrada corrida 1 valor hacia el futuro
dataset = dataset.window(window_length, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))

batch_size = 256
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1,0], windows[:, 1:,0])) # (x, y)

# Las entradas categoricas usualmente se codifican, como one-hot o usando embedding. Probar de no hacerlo
dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [12]:
#visualizo algunos datos. Las entradas son secuencias one-hot, y las salidas secuencias numericas! 
for i in dataset.take(1):
    print(i[1].shape)

2021-11-13 14:32:57.303905: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-11-13 14:32:57.322350: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 3199980000 Hz


(256, 100)


### Modelo:
Para hacer las predicciones vamos a armar un modelo RNN usando 2 capas GRU de 128 neuronas, con un 20% de Dropout en las entradas como en los estados ocultos. porque? porque funciona, y es un buen punto inicial. La capa de salida sera una capa Densa distribuida en el tiempo como siempre en estos modelos, que en este caso tenrdra tantas neuronas como caracteres tiene nuestro dataset, ya que la salida es la probabilidad que tiene cada caracter de ser el siguiente. Como la salida de probabilidad debe sumar 1, se aplica la activación softmax a la salida de esta capa densa. Como se predicen probabilidades, se compila con la loss "sparse_categorical_crossentropy".

In [17]:
model = tf.keras.models.Sequential([
    tfkl.GRU(128, return_sequences=True, input_shape=[None, max_id], dropout=0.2),
    tfkl.GRU(128, return_sequences=True, dropout=0.2),
    tfkl.TimeDistributed(tfkl.Dense(max_id, activation="softmax"))
    ])

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=20)

Epoch 1/20
310/310 [==============================] - 13s 35ms/step - loss: 2.3326
Epoch 2/20
310/310 [==============================] - 11s 35ms/step - loss: 1.9212
Epoch 3/20
310/310 [==============================] - 11s 35ms/step - loss: 1.7369
Epoch 4/20
310/310 [==============================] - 11s 35ms/step - loss: 1.6139
Epoch 5/20
310/310 [==============================] - 11s 35ms/step - loss: 1.5231
Epoch 6/20
310/310 [==============================] - 12s 35ms/step - loss: 1.4505
Epoch 7/20
310/310 [==============================] - 12s 35ms/step - loss: 1.3954
Epoch 8/20
310/310 [==============================] - 12s 35ms/step - loss: 1.3500
Epoch 9/20
310/310 [==============================] - 12s 35ms/step - loss: 1.3138
Epoch 10/20
310/310 [==============================] - 12s 35ms/step - loss: 1.2825
Epoch 11/20
310/310 [==============================] - 12s 35ms/step - loss: 1.2561
Epoch 12/20
310/310 [==============================] - 12s 35ms/step - loss: 1.2338
E

### Usando el modelo
Hay que hacer un pre-procesamiento para usar este modelo. Lo mas simple seria tomar una entrada de texto, hacer el one-hot encoding, pasarlo al modelo para hacer prediccion (el modelo recibe secuencias de cualquier largo!), tomar la salida del modelo (que es un numero, un codigo o ID) y convertirla a caracter a traves del tokenizer.

In [18]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

X_new = preprocess(["Holo es"])
Y_pred = model.predict_classes(X_new)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1]

't'

# Generar Texto!
Al igual que en los ejemplos de series numéricas, puedo predecir varios valores futuros yendo de uno a la vez, y sumando este valor al cuerpo de valores de entrada a medida que hago predicciones. En generación de texto esta metodología lleva a repetir palabras u oraciones. Otra alternativa mejor es elegir el caracter siguiente de manera aleatoria, con una probabilidad igual a la estimada por el modelo. Esto genera texto mas diverso e interesante.
Para hacer esto, dependemos de la función **tf.random.categorical()**. Esta funcion devuelve muestras random de un cierto numero de clases, dado las probabilidades de cada clase (logits). Para tener mas control sobre la diversidad del texto generado, se pueden dividir los logits por un número denominado "temperatura" que se puede ajustar. Con un valor de temperatura = 0, se favorecen los caracteres de alta probabilidad, y con una temperatura cerca de 1 se les da a todos el mismo peso. 

In [19]:
def simple_gen(text, n_chars):    
    for _ in tqdm(range(n_chars)):
        X_new = preprocess([text])
        Y_pred = model.predict_classes(X_new)
        next_char = tokenizer.sequences_to_texts(Y_pred + 1)[0][-1]
        text += next_char
    return text

In [20]:
print(simple_gen("rio", 200))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:06<00:00, 32.61it/s]

rio  
con la entereza de un gran titan    
muero contento hemos batido   
al enemigo che capitan   


con su figura doblada   
vencida como la tarde  


por eso es que mi cancion  


por eso es que mi ca


In [21]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

def complete_text(text, n_chars=200, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [24]:
print(complete_text("rio ", temperature=0.25))

rio  
correntino este donde este  


se que tu comprendera  


por falos del taragi    
solor de habra de llamar   
roco en el rostro  
de los pobres pescadores  


por eso es que mi cancion  


por eso e
